In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import json
import urllib
import cobra

In [ ]:
#bigg_universal = cobra.io.load_json_model(urllib.request.urlopen('http://bigg.ucsd.edu/static/namespace/universal_model.json'))
#cobra.io.save_json_model(bigg_universal, "/Users/kimj972/Documents/Research/Rhodosporidium_toruloides/Metabolic_model/BiGG_universal.json")

In [ ]:
# bigg_universal = cobra.io.load_json_model("/Users/kimj972/Documents/Research/Rhodosporidium_toruloides/Metabolic_model/BiGG_universal.json")
bigg_universal = cobra.io.load_json_model("GSMs/universal_model.json")

In [ ]:
# model = cobra.io.load_json_model("/Users/kimj972/Documents/Research/Rhodosporidium_toruloides/Metabolic_model/IFO0880_GPR_2e.json")
model = cobra.io.read_sbml_model("carveme/Ropacus_carveme_raw.xml")

Joonhoon's version had no output<br>
sorted function seems unneccesary

First check that all genes have at least one reaction.<br>
Then check that all metabolites have at least one reaction.<br>
Orphan gene/metabolite check

In [ ]:
print(len(model.genes))
print(model.genes[0])
print(model.genes[0].reactions)

In [ ]:
for x in sorted(model.genes, key=lambda x: x.id):
    if not x.reactions:
        print(x)
print()
for x in sorted(model.metabolites, key=lambda x: x.id):
    if not x.reactions:
        print(x)

# Metabolite annotation

JH's version printed the notes for all metabolites

In [ ]:
for m in model.metabolites[:25]:
    if m.notes:
        print(m, m.notes)

My model: Metabolite notes contain formula, and occassionally SEED Compound, UniPathway Compound, and KEGG Compound is included.<br>
JH's model: Every metabolite has 'original_bigg_ids' information

check original_bigg_ids for met and rxn

In [ ]:
for m in model.metabolites[:25]:
    if m.annotation:
        print(m, m.annotation)

My model doesn't return anything here <br>
JH's model returns annotation for 6 out of all of the metabolites

In [ ]:
set(sum([list(m.annotation.keys()) for m in model.metabolites],[]))

This function gives the set of all the keys found in the notes dictionary of model metabolites <br>
JH's output: <br> {'bigg.metabolite',
 'biocyc',
 'chebi',
 'envipath',
 'hmdb',
 'inchi_key',
 'kegg.compound',
 'kegg.drug',
 'metanetx.chemical',
 'reactome.compound',
 'sabiork',
 'sbo',
 'seed.compound'} <br>
 

In [ ]:
bigg_universal.metabolites.get_by_id('glc__D_c').annotation

Example of universal model annotation. <br>
One problem is that there are multiple annotations for a single field

In [ ]:
print(type(     bigg_universal.metabolites.get_by_id('glc__D_c').annotation))
print(type(dict(bigg_universal.metabolites.get_by_id('glc__D_c').annotation)))

this problem is solved if you cast the list of lists to a dict

In [ ]:
set(sum([[x[0] for x in m.annotation] for m in bigg_universal.metabolites],[]))

Get the set of all keys used in metabolite annotations from the bigg universal model

In [ ]:
for m in model.metabolites:
    if m.id in bigg_universal.metabolites:
        #m.notes.update(dict(bigg_universal.metabolites.get_by_id(m.id).notes))
        #m.annotation.update(dict(bigg_universal.metabolites.get_by_id(m.id).annotation))
        m.annotation = dict(bigg_universal.metabolites.get_by_id(m.id).annotation)

This is the proper way to annotate metabolites using the universal model so that MEMOTE will be happy

In [ ]:
m_annotation = dict({'BioCyc': 'biocyc',
                     'CHEBI': 'chebi',
                     'Human Metabolome Database': 'hmdb',
                     'InChI Key': 'inchikey',
                     'KEGG Compound': 'kegg.compound',
                     'KEGG Drug': 'kegg.drug',
                     'KEGG Glycan': 'kegg.glycan',
                     'LipidMaps': 'lipidmaps',
                     'MetaNetX (MNX) Chemical': 'metanetx.chemical',
                     'Reactome Compound': 'reactome',
                     'SEED Compound': 'seed.compound'})

We still need to use the proper key names as provided in this dictionary

In [ ]:
for m in model.metabolites:
    if m.annotation:
        m.annotation = dict((m_annotation[k], v.rsplit('/',1)[-1]) for k, v in m.annotation.items())
    m.annotation['bigg.metabolite'] = m.id

Loop through metabolites: <br>
>if the metabolite has an annotation: <br>
        >>then set the annotation using the following:<br>
        a dictionary that is populated by a list comprehension that loop over the current dictionary's items<br>
        each time the key and value from the current dictionary are return as k,v <br>
        the new dictionary's key is returned using the m_annotation dictionary <br>
        the new dictionary's value is returned by spliting the url from the existing dictionary's value at all '/' characters<br>
        and returning the last one


In [ ]:
for m in model.metabolites:
    m.annotation['sbo'] = 'SBO:0000247'

All metabolites are given the [systems biology ontology id 247](https://www.ebi.ac.uk/sbo/main/SBO:0000247), indicating that they are simple chemicals

In [ ]:
model.metabolites.get_by_id('glc__D_c').annotation

check annotation dictionary for an arbitrary metabolite

In [ ]:
model.metabolites.get_by_id('12dgr120_c').annotation

In [ ]:
model.metabolites.get_by_id('12dgr120_c').name

# Reaction annotation

In [ ]:
for r in sorted(model.reactions[:100], key=lambda x: x.id):
    if r.notes:
        print(r, r.notes)

My model has some kegg and some biopath reaction annotations
JH's model has 'original bigg ids' reaction annotation

In [ ]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.annotation:
        print(r, r.annotation)

Both mine and JH's models originally have no reaction annotations

In [ ]:
for r in sorted(model.reactions, key=lambda x: x.id):
    r.annotation = {}

Ensure that all reactions have blank annotations

In [ ]:
for r in model.reactions:
    if r.boundary:
        if r.id.startswith('DM_') or r.id.startswith('SRC_'):
            r.annotation = {'SBO': 'SBO:0000628'}
        elif r.id.startswith('EX_'):
            r.annotation = {'SBO': 'SBO:0000627'}
        elif r.id.startswith('SK_'):
            r.annotation = {'SBO': 'SBO:0000632'}

r.boundary is a boolean that returns true when a reaction is transport related<br>

Systems biology ontology definitions:<br>
SBO:0000628 : [demand reaction](http://www.ebi.ac.uk/sbo/main/SBO:0000628) <br>
SBO:0000627 : [exchange reaction](http://www.ebi.ac.uk/sbo/main/SBO:0000627) <br>
SBO:0000632 : [sink reaction](http://www.ebi.ac.uk/sbo/main/SBO:0000632) <br>


In [ ]:
for r in model.reactions[:10]:
    if 'C_e' in r.compartments:
        print(r, r.compartments)

r.compartments returns a set of compartments labeled <br>
cytoplasm: 'C_c' <br>
periplasm: 'C_p' <br>
extracellular: 'C_e' <br>

In [ ]:
for r in model.reactions[:100]:
    if len(r.compartments) > 1 and 't' in r.id:
        print(r)

Not sure why the 't' in id portion is included
Prints all reactions that take place between compartments

In [ ]:
for r in sorted(model.reactions[:500], key=lambda x: x.id):
    if (len(r.compartments) > 1 
        and not 't' in r.id 
        and not 'd' in r.compartments 
        and not any(x in [m.id for m in r.metabolites] for x in ['focytc_m','q9_m','fadh2_m','dolmanp_r','pc_RT_r'])):
        print(r)

Prints reactions that take place in at least 2 compartments , not including any reaction in compartment d. <br>
Also excludes reactions that contain these metabolites: 'focytc_m','q9_m','fadh2_m','dolmanp_r','pc_RT_r' <br>

Not sure where that metabolite list comes from


In [ ]:
for r in model.reactions:
    if len(r.compartments) > 1 and 't' in r.id:
        r.annotation['SBO'] = 'SBO:0000655'
    if (len(r.compartments) > 1 and not 't' in r.id and not ('d' in r.compartments or
        any(x in [m.id for m in r.metabolites] for x in ['focytc_m','q9_m','fadh2_m','dolmanp_r','pc_RT_r']))):
        r.annotation['SBO'] = 'SBO:0000655'

Systems biology ontology definitions:<br>
SBO:0000655 : [transport reaction, definition: The movement of an entity/entities across a biological membrane mediated by a transporter protein.](http://www.ebi.ac.uk/sbo/main/SBO:0000655) <br>

In [ ]:
for r in model.reactions:
    if len(r.compartments) == 1 and not r.boundary:
        r.annotation['SBO'] = 'SBO:0000176'
    if (len(r.compartments) > 1 and not 't' in r.id and ('d' in r.compartments or
        any(x in [m.id for m in r.metabolites] for x in ['focytc_m','q9_m','fadh2_m','dolmanp_r','pc_RT_r']))):
        r.annotation['SBO'] = 'SBO:0000176'

Systems biology ontology definitions:<br>
SBO:0000176 : [biochemical reaction, definition: An event involving one or more chemical entities that modifies the electrochemical structure of at least one of the participants.](http://www.ebi.ac.uk/sbo/main/SBO:0000176) <br>

In [ ]:
model.reactions.get_by_id('Growth').annotation['SBO'] = 'SBO:0000629'

Systems biology ontology definitions:<br>
SBO:0000629 : [biomass production](http://www.ebi.ac.uk/sbo/main/SBO:0000629) <br>

In [ ]:
set(sum([list(r.annotation.keys()) for r in model.reactions],[]))

JH and my model both only have 'SBO'

In [ ]:
for r in model.reactions:
    if r.id in bigg_universal.reactions:
        #r.notes.update(dict(bigg_universal.reactions.get_by_id(r.id).notes))
        r.annotation.update(dict(bigg_universal.reactions.get_by_id(r.id).annotation))

In [ ]:
set(sum([list(r.annotation.keys()) for r in model.reactions],[]))

JH also has: {'BioCyc',
 'EC Number',
 'KEGG Reaction',
 'MetaNetX (MNX) Equation',
 'RHEA',
 'Reactome Reaction',
 'SBO',
 'SEED Reaction'}

In [ ]:
model.reactions.get_by_id('PGI').annotation

An arbitrarily chosen reaction's annotation

In [ ]:
# Reformat to memote style
r_annotation = dict({'BioCyc': 'biocyc',
                     'EC Number': 'ec-code',
                     'KEGG Reaction': 'kegg.reaction',
                     'MetaNetX (MNX) Equation': 'metanetx.reaction',
                     'RHEA': 'rhea',
                     'Reactome Reaction': 'reactome',
                     'SBO': 'sbo',
                     'SEED Reaction': 'seed.reaction'})

Proper dictionary keys to annotate a reaction according to MEMOTE

In [ ]:
for r in model.reactions:
    if r.annotation:
        r.annotation = dict((r_annotation[k], v.rsplit('/',1)[-1]) for k, v in r.annotation.items())
    r.annotation['bigg.reaction'] = r.id

In [ ]:
model.reactions.get_by_id('PGI').annotation

In [ ]:
set(r.subsystem for r in model.reactions)

JH has long list of subsystems here: the ones on kegg: <br>
{'',
 'Alanine and Aspartate Metabolism',
 'Alanine and aspartate metabolism',
 'Aldehyde degradation',
 'Alkaloid biosynthesis II',
 'Alternate Carbon Metabolism',
 'Amino sugar and nucleotide sugar metabolism',
 'Aminosugar Metabolism',
 'Anaplerotic Reactions',
 'Arginine and Proline Metabolism',
 'Arginine and proline metabolism',
 'Ascorbate metabolism',
 'B-Ketoadipate pathway',
 'Bile Acid Biosynthesis',
 'Biomass and maintenance functions',
 'Biosynthesis of steroids',
 'Biosynthesis of steroids: Terpenoid backbone synthesis',
 'Biosynthesis of steroids: Ubiquinone/Terpenoid-quinone biosynthesis',
 'Biosynthesis of unsaturated fatty acids',
 'Biotin Metabolism',
 'Butanoate Metabolism',
 'Butanoate metabolism',
 'Calvin cycle/Pentose phosphate pathway',
 'Carbon fixation',
 'Carbon fixation;Glutamate metabolism;Alanine and aspartate metabolism',
 'Carnitine shuttle',
 'Cell Envelope Biosynthesis',
 'Cholesterol Metabolism',
 'Citric Acid Cycle',
 'Cofactor and Prosthetic Group Biosynthesis',
 'Cofactor biosynthesis: Carnitine biosynthesis',
 'Cofactor recycling',
 'Cysteine Metabolism',
 'Cysteine and methionine metabolism',
 'Cysteine metabolism',
 'Drug metabolism - other enzymes',
 'Eicosanoid Metabolism',
 'Extracellular exchange',
 'Fatty Acid Metabolism',
 'Fatty acid activation',
 'Fatty acid elongation',
 'Fatty acid elongation in mitochondria',
 'Fatty acid metabolism',
 'Fatty acid oxidation',
 'Fatty acid oxidation, peroxisome',
 'Fatty acid synthesis',
 'Folate Metabolism',
 'Folate biosynthesis',
 'Fructose and Mannose Metabolism',
 'Galactose metabolism',
 'Glutamate Metabolism',
 'Glutamate metabolism',
 'Glutathione Metabolism',
 'Glycerolipid metabolism',
 'Glycerophospholipid Metabolism',
 'Glycerophospholipid metabolism',
 'Glycine and Serine Metabolism',
 'Glycine, Serine and threonine metabolism',
 'Glycine, Serine, and Threonine Metabolism',
 'Glycine, serine and threonine metabolism',
 'Glycine, serine, alanine, and threonine metabolism',
 'Glycolysis',
 'Glycolysis / Gluconeogenesis',
 'Glycolysis/Gluconeogenesis',
 'Glyoxylate Metabolism',
 'Glyoxylate and Dicarboxylate Metabolism',
 'Glyoxylate metabolism',
 'High-mannose type N-glycan biosynthesis',
 'Histidine Metabolism',
 'Histidine metabolism',
 'Homogentisate Pathway',
 'Inorganic Ion Transport and Metabolism',
 'Inositol Phosphate Metabolism',
 'Inositol phosphate metabolism',
 'Intracellular demand',
 'Lipopolysaccharide Biosynthesis / Recycling',
 'Lysine Metabolism',
 'Lysine biosynthesis',
 'Membrane Lipid Metabolism',
 'Metabolite Repair',
 'Methionine Metabolism',
 'Methionine and cysteine metabolism',
 'Methionine metabolism',
 'Methylglyoxal Metabolism',
 'Murein Recycling',
 'N-Glycan Biosynthesis',
 'N-Glycan Degradation',
 'N-Glycan biosynthesis',
 'NAD Metabolism',
 'Nicotinate and nicotinamide metabolism',
 'Nitrogen Metabolism',
 'Nitrogen metabolism',
 'Nucleotide Salvage Pathway',
 'Nucleotide Sugar Metabolism',
 'Nucleotide interconversion',
 'Nucleotides',
 'One carbon pool by folate',
 'Oxidative Phosphorylation',
 'Oxidative phosphorylation',
 'Pantothenate and CoA biosynthesis',
 'Pentose Phosphate Pathway',
 'Pentose and Glucuronate Interconversions',
 'Pentose phosphate pathway',
 'Phenylalanine metabolism',
 'Polyamine metabolism',
 'Porphyrin and Chlorophyll Metabolism',
 'Porphyrin and chlorophyll metabolism',
 'Propanoate Metabolism',
 'Propanoate metabolism',
 'Purine Catabolism',
 'Purine Metabolism',
 'Purine and Pyrimidine Biosynthesis',
 'Purine metabolism',
 'Pyrimidine Catabolism',
 'Pyrimidine metabolism',
 'Pyruvate Metabolism',
 'Pyruvate metabolism',
 'ROS Detoxification',
 'Riboflavin Metabolism',
 'S_Alanine_and_Aspartate_Metabolism',
 'S_Alternate_Carbon_Metabolism',
 'S_Anaplerotic_reactions',
 'S_Arabinose_Metabolism',
 'S_Arginine_and_Proline_Metabolism',
 'S_Asparagine_metabolism',
 'S_Carbohydrates_and_related_molecules',
 'S_Citric_Acid_Cycle',
 'S_Complex_Alcohol_Metabolism',
 'S_Cysteine_Metabolism',
 'S_Fatty_Acid_Degradation',
 'S_Fatty_Acid_Metabolism',
 'S_Fatty_Acid__Biosynthesis',
 'S_Folate_Metabolism',
 'S_Fructose_and_Mannose_Metabolism',
 'S_Galactose_metabolism',
 'S_Glutamate_metabolism',
 'S_Glutamine_Metabolism',
 'S_Glycerolipid_Metabolism',
 'S_Glycine_and_Serine_Metabolism',
 'S_GlycolysisGluconeogenesis',
 'S_Glycoprotein_Metabolism',
 'S_Histidine_Metabolism',
 'S_Methane_Metabolism',
 'S_Methionine_Metabolism',
 'S_NAD_Biosynthesis',
 'S_Nitrogen_Metabolism',
 'S_Nucleotide_Salvage_Pathway',
 'S_Other',
 'S_Other_Amino_Acid_Metabolism',
 'S_Oxidative_Phosphorylation',
 'S_Pantothenate_and_CoA_Biosynthesis',
 'S_Pentose_Phosphate_Pathway',
 'S_Phospholipid_Biosynthesis',
 'S_Phospholipid_Metabolism',
 'S_Porphyrin_and_Chlorophyll_Metabolism',
 'S_Purine_and_Pyrimidine_Biosynthesis',
 'S_Pyridoxine_Metabolism',
 'S_Pyruvate_Metabolism',
 'S_Quinone_Biosynthesis',
 'S_Riboflavin_Metabolism',
 'S_Sphingolipid_Metabolism',
 'S_Starch_and_Sucrose_Metabolism',
 'S_Sterol_Metabolism',
 'S_Thiamine_Metabolism',
 'S_Threonine_and_Lysine_Metabolism',
 'S_Transport__Endoplasmic_Reticular',
 'S_Transport__Extracellular',
 'S_Transport__Golgi_Apparatus',
 'S_Transport__Mitochondrial',
 'S_Transport__Nuclear',
 'S_Transport__Peroxisomal',
 'S_Transport__Vacuolar',
 'S_Tyrosine__Tryptophan__and_Phenylalanine_Metabolism',
 'S_Valine__Leucine__and_Isoleucine_Metabolism',
 'S_Xylose_Metabolism',
 'S_tRNA_charging',
 'Selenoamino acid metabolism',
 'Starch and Sucrose Metabolism',
 'Starch and sucrose metabolism',
 'Steroid Metabolism',
 'Sulfur Metabolism',
 'Sulfur metabolism',
 'TCA cycle;Carbon fixation;CO2 fixation',
 'Taurine and hypotaurine metabolism',
 'Terpenoid backbone biosynthesis',
 'Tetrahydrobiopterin',
 'Threonine and Lysine Metabolism',
 'Transport',
 'Transport, Endoplasmic Reticular',
 'Transport, Extracellular',
 'Transport, Golgi Apparatus',
 'Transport, Inner Membrane',
 'Transport, Mitochondrial',
 'Transport, Nuclear',
 'Transport, Outer Membrane Porin',
 'Transport, Peroxisomal',
 'Transport, endoplasmic reticular',
 'Transport, extracellular',
 'Transport, glyoxysome',
 'Transport, mitochondria',
 'Transport, mitochondrial',
 'Transport, nuclear',
 'Transport, nucleus',
 'Transport, peroxisomal',
 'Tryptophan metabolism',
 'Tyrosine metabolism',
 'Tyrosine, Tryptophan, and Phenylalanine Metabolism',
 'Ubiquinone synthesis',
 'Unassigned',
 'Urea cycle/amino group metabolism',
 'Valine, Leucine, and Isoleucine Metabolism',
 'Valine, leucine and isoleucine degradation',
 'Valine, leucine, and isoleucine metabolism',
 'Vitamin B2 metabolism',
 'Vitamin B6 Metabolism',
 'Vitamin B6 metabolism',
 'beta-Alanine metabolism',
 'beta-Alanine metabolism;Pyrimidine metabolism',
 'tRNA Charging'}

In [ ]:
for r in model.reactions:
    if r.subsystem.startswith('S_'):
        r.subsystem = r.subsystem.replace('S_','').replace('__',', ').replace('_',' ')
    r.subsystem = r.subsystem.replace('Glycolysis / Gluconeogenesis','Glycolysis/Gluconeogenesis')
    r.subsystem = r.subsystem.replace('GlycolysisGluconeogenesis','Glycolysis/Gluconeogenesis')
    r.subsystem = r.subsystem.replace('metabolism','Metabolism')
    r.subsystem = r.subsystem.replace('aspartate','Aspartate')
    r.subsystem = r.subsystem.replace('reactions','Reactions')
    r.subsystem = r.subsystem.replace('proline','Proline')
    r.subsystem = r.subsystem.replace('serine','Serine')
    r.subsystem = r.subsystem.replace('Serine and threonine','Serine, and Threonine')
    r.subsystem = r.subsystem.replace(', alanine',', Alanine')
    r.subsystem = r.subsystem.replace('phosphate','Phosphate')
    r.subsystem = r.subsystem.replace('pathway','Pathway')
    r.subsystem = r.subsystem.replace('biosynthesis','Biosynthesis')
    r.subsystem = r.subsystem.replace('chlorophyll','Chlorophyll')
    r.subsystem = r.subsystem.replace('sucrose','Sucrose')
    r.subsystem = r.subsystem.replace('endoplasmic reticular','Endoplasmic Reticular')
    r.subsystem = r.subsystem.replace('extracellular','Extracellular')
    r.subsystem = r.subsystem.replace('Transport, mitochondria','Transport, Mitochondrial')
    r.subsystem = r.subsystem.replace('Transport, mitochondrial','Transport, Mitochondrial')
    r.subsystem = r.subsystem.replace('peroxisomal','Peroxisomal')
    r.subsystem = r.subsystem.replace('leucine and isoleucine','Leucine, and Isoleucine')
    r.subsystem = r.subsystem.replace('leucine, and isoleucine','Leucine, and Isoleucine')
    r.subsystem = r.subsystem.replace('charging','Charging')
    r.subsystem = r.subsystem.replace('phosphorylation','Phosphorylation')
    r.subsystem = r.subsystem.replace('degradation','Degradation')

acceptable formatting for subsystems

In [ ]:
temp = [r.subsystem for r in model.reactions]
for k in sorted(set(temp)):
    print(k, temp.count(k))

Prints the name of each subsystem in the model and the number of times it is used

In [ ]:
for r in sorted(model.reactions[:5], key=lambda x: x.id):
    if not r.subsystem:
        print(r)
        for m in r.metabolites:
            if len(m.reactions) < 7:
                print([(r2.id, r2.subsystem) for r2 in m.reactions if r2 is not r])

If a reaction does not have a subsystem, then go through metabolites. <br>
If the metabolite is involved in less than 7 reactions, then print the reaction id and subsystem for reactions that are not the one in question

In [ ]:
model.reactions.get_by_id('34DHPHAMT').subsystem = 'Tyrosine Metabolism'
model.reactions.get_by_id('3DSPHRer').subsystem = 'Sphingolipid Metabolism'
model.reactions.get_by_id('42A12BOOX').subsystem = 'Tyrosine Metabolism'
model.reactions.get_by_id('ABUTD').subsystem = 'Arginine and Proline Metabolism'
model.reactions.get_by_id('ACACT4m').subsystem = 'Fatty acid elongation in mitochondria'
model.reactions.get_by_id('ACACT6m').subsystem = 'Fatty acid elongation in mitochondria'
model.reactions.get_by_id('ACACT7m').subsystem = 'Fatty acid elongation in mitochondria'
model.reactions.get_by_id('ACOAH').subsystem = 'Pyruvate metabolism'
model.reactions.get_by_id('ACONTam').subsystem = 'Citric Acid Cycle'
model.reactions.get_by_id('ACONTbm').subsystem = 'Citric Acid Cycle'
model.reactions.get_by_id('ALDD20x').subsystem = 'Tyrosine, Tryptophan, and Phenylalanine Metabolism'
model.reactions.get_by_id('AMAOTrm').subsystem = 'Pantothenate and CoA Biosynthesis'
model.reactions.get_by_id('AMETt2').subsystem = 'Transport, Extracellular'
model.reactions.get_by_id('ASP1DC').subsystem = 'beta-Alanine Metabolism'
model.reactions.get_by_id('LEUTRSm').subsystem = 'Valine, Leucine, and Isoleucine Metabolism'
model.reactions.get_by_id('PYDXO_1').subsystem = 'Vitamin B6 Metabolism'

manually add subsystems to reactions

# Gene Annotation

In [ ]:
for g in sorted(model.genes, key=lambda x: x.id):
    if g.annotation:
        print(g)

In [ ]:
for g in sorted(model.genes[:10], key=lambda x: x.id):
    if g.name:
        print(g, g.name, g.reactions)

print gene, gene name, and reactions associated with gene

In [ ]:
Uniprot = pd.read_csv('/Users/kimj972/Documents/Research/Rhodosporidium_toruloides/Metabolic_model/IFO0880_4_Uniprot.txt',
                      sep='\t', index_col=0)
Uniprot.index = Uniprot.index.map(str)
Uniprot

output should look something like this: <br>
    	UniProt
RTO4_ID	<br>
10632	B3TQJ2<br>
10637	B3TQJ2<br>
10640	B3TQJ2<br>
15931	A0A2T0A585<br>
15937	A0A2T0A585<br>

uniprot is a dataframe with index that comes from mapping the original index through the str() function

In [ ]:
for g in model.genes:
    if g.id not in Uniprot.index:
        print(g)

JH has about 12 genes that are not in uniprot

In [ ]:
for g in model.genes:
    if g.id in Uniprot.index:
        g.annotation['uniprot'] = Uniprot.loc[g.id,'UniProt']
        g.annotation['jgi'] = 'Rhoto_IFO0880_4_'+g.id
        g.annotation['ncbiprotein'] = 'AAT19DRAFT_'+g.id
        g.name = 'jgi|Rhoto_IFO0880_4|'+g.id     

if the gene is in uniprot, then update the gene annotation and name with the uniprot id

In [ ]:
for g in model.genes:
    g.annotation['sbo'] = 'SBO:0000243'

Systems biology ontology definitions:<br>
SBO:0000243 : [gene](http://www.ebi.ac.uk/sbo/main/SBO:0000243) <br>

In [ ]:
for x in sorted(model.genes, key=lambda x: x.id):
    if not x.reactions:
        print(x)
print()
for x in sorted(model.metabolites, key=lambda x: x.id):
    if not x.reactions:
        print(x)

In [ ]:
cobra.manipulation.remove_genes(model, [x for x in model.genes if not x.reactions])
model.remove_metabolites([x for x in model.metabolites if not x.reactions])

remove genes and metabolites that are not in reactions

In [ ]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if not r.boundary and sum(abs(x) for x in r.check_mass_balance().values()) > 1e-12:
        print(r, r.gene_reaction_rule, r.check_mass_balance())

print reaction, gpr, and check mass balance if a reaction is not a boundry and the mass balance value is greater than 1e-12

In [ ]:
print(len(model.genes))
print(len(model.reactions))
print(len(model.metabolites))
print(len(set([m.id.rsplit('_',1)[0] for m in model.metabolites])))
print(len(model.compartments))
model

In [ ]:
model.id = 'Rt_IFO0880'
model.name = 'Rt_IFO0880'
model.description = 'R. toruloides IFO0880'

update model meta info

In [ ]:
model

In [ ]:
cobra.io.save_json_model(model, "/Users/kimj972/Documents/Research/Rhodosporidium_toruloides/Metabolic_model/Rt_IFO0880.json")

In [ ]:
cobra.io.write_sbml_model(model, "/Users/kimj972/Documents/Research/Rhodosporidium_toruloides/Metabolic_model/Rt_IFO0880.xml")

In [ ]:
cobra.io.save_matlab_model(model, "/Users/kimj972/Documents/Research/Rhodosporidium_toruloides/Metabolic_model/Rt_IFO0880.mat")

Save model as json, sbml and matlab

In [ ]:
%%bash
source activate python3_cobrapy
cd /Users/kimj972/Documents/Research/Rhodosporidium_toruloides/Metabolic_model
memote report snapshot --solver cplex Rt_IFO0880.xml
mv index.html Rt_IFO0880.html

open bash terminal <br>
activate virtual environment <br>
change directory <br>
run memote using cplex on annotated model <br>
change name of output to more appropriate name
